In [1]:
import sys
sys.path.insert(0, "../")
sys.path.insert(0, "../dataset_generation/")

import json
import torch
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt

#plt.style.use("project_style.mplstyle")

import stats
import scipy.stats
from tqdm import tqdm
import analysis_tools as tools

In [2]:
sequences = 100
timesteps = 500

#sequence = ["square", "walled_square"]
sequence = ["square", "square"]

res = 16
bins = [[res, res] for i in range(len(sequence))]

In [3]:
# load model and specs
model_name = "al1_10_l2_1"
path = f"../models/{model_name}"
model, params = tools.load_model(path, device = "cpu", model_type = "RNN")

In [4]:
model

VPC_RNN(
  (g): RNN(8, 500, bias=False, batch_first=True)
  (p): Linear(in_features=500, out_features=100, bias=False)
  (activation): ReLU()
  (loss_fn): MSELoss()
)

In [5]:
xa, ra, va, ca = tools.test_dataset(sequences*[sequence[0]], timesteps, context = params["context"], 
                                    device = "cpu", trajectories = True)
xb, rb, vb, cb = tools.test_dataset(sequences*[sequence[1]], timesteps, context = params["context"], 
                                    device = "cpu", trajectories = True)
ra.shape, va.shape, ca.shape

((100, 500, 2), (100, 500, 2), (100, 500, 6))

In [6]:
gs = []
ps = []
g_prev = None
for x in (xa, xb):
    yhat, g, p, mu = model(x, g_prev = g_prev)
    g_prev = g[:,-1]
    gs.append(g.detach().numpy())
    ps.append(p.detach().numpy())

gs = np.concatenate(gs, axis = 1)
ps = np.concatenate(ps, axis = 1)
rs = np.concatenate([ra, rb], axis = 1)
gs.shape, ps.shape, rs.shape

((100, 1000, 500), (100, 1000, 100), (100, 1000, 2))

In [7]:
g_ratemaps = np.zeros((2*timesteps, params["nodes"], res, res)).astype("float32")
p_ratemaps = np.zeros((2*timesteps, params["outputs"], res, res)).astype("float32")

In [8]:
def autocorr(x):
    z = np.zeros((len(x), len(x)))
    for i in tqdm(range(len(x))):
        for j in range(len(x)):
            z[i, j] = np.corrcoef(x[i], x[j])[1,0]
    return z

In [9]:
for i in range(2*timesteps):
    p_ratemaps[i] = stats.population_vector_ratemaps(ps[None,:,i], rs[None,:,i], bins, smooth = True)
    g_ratemaps[i] = stats.population_vector_ratemaps(gs[None,:,i], rs[None,:,i], bins, smooth = True)

In [ ]:
g_corr = autocorr(g_ratemaps.reshape((g_ratemaps.shape[0], -1)))

 65%|█████████████████████████▊              | 646/1000 [15:56<08:43,  1.48s/it]

In [ ]:
p_corr = autocorr(p_ratemaps.reshape((p_ratemaps.shape[0], -1)))

In [ ]:
plt.imshow(g_corr, cmap = "Blues")
plt.colorbar()

In [ ]:
plt.imshow(p_corr, cmap = "coolwarm")
plt.colorbar()